In [2]:
import pickle
from typing import List, Dict
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import geopy.distance

In [38]:
pd.set_option('display.max_columns', None)

In [3]:
DATA = '../data/'
KAG = DATA + 'kaggle_data/'

teams = pd.read_csv(KAG + 'MTeams.csv')
reg = pd.read_csv(KAG + 'MRegularSeasonCompactResults.csv')
post = pd.read_csv(KAG  + 'MNCAATourneyCompactResults.csv')
cities = pd.read_csv(KAG + 'Cities.csv')
gameCities = pd.read_csv(KAG + 'MGameCities.csv')
ords = pd.read_csv(KAG + 'MMasseyOrdinals.csv')

## 1. Construct 'favorite' model

### Description: 

Set of games considered:
All regular season games

Inputs used:
Rating systems + Distance to games

### Why favored:

-Lots of data - Rating systems and results go back to 2003

-"Compact" variables - ratings systems are designed to capture a lot of what we would already be trying to model if we were to feature engineer ourselves from the raw data

### 1a. Get results first

In [40]:
assert all(post.columns == reg.columns)
all_games = pd.concat([reg, post])
all_games['wins'] = 1
all_games.rename(columns = {'WTeamID':'Tm1ID', 
                            'LTeamID':'Tm2ID',
                            'WScore' : 'Tm1Score', 
                            'LScore' : 'Tm2Score', 
                            'WLoc' : 'Tm1Loc'}, inplace = True)

In [41]:
"""
=======================
A. Merge on game cities
=======================
"""
def mergeGameCities(df, gameCities, cities):
    """
    1. Merge on game cities
    """
    # a. Merge
    res = df.merge(gameCities, left_on = ['Season','DayNum','Tm1ID','Tm2ID'],
                                    right_on = ['Season','DayNum','WTeamID','LTeamID'],
                                    how = 'outer',
                                    indicator= 'has_city')
    # b. Check that after merging game cities
    #     that entire dataset has NCAA Tournament or regular season games
    assert len(res.loc[res['has_city']== 'right_only','CRType'].value_counts()) == 1
    assert len(res.loc[res['has_city'] != 'right_only','CRType'].value_counts()) == 2

    # c. Check city merge result is complete for data from 2010 onwards
    assert len(df[df['Season'] >= 2010]) == len(res[res['has_city'] == 'both'])

    # d. Reduce to left-only, drop vars, update city merge indicator
    res = res[res['has_city'] != 'right_only']
    res['has_city'] = np.where(res['has_city'] == 'both', 1,0)

    """
    2. Merge on actual city information
    """
    # a. Merge
    res = res.merge(cities, on = 'CityID', how = 'left', indicator = True)

    # b. Check that everyone that had city in last merge also got real city info
    assert (res.loc[res['has_city'] == 1, '_merge'] == 'both').mean() == 1

    # c. Drop extraneous columns
    res.drop(['WTeamID','LTeamID','_merge'],axis = 1, inplace = True)
    return res

all_games = mergeGameCities(all_games, gameCities, cities)

In [42]:
"""
==================
A. Copy and stack
==================
"""
all_games_c = all_games.copy()
all_games_c.rename(columns = {'Tm1ID' : 'Tm2ID', 
                            'Tm2ID' : 'Tm1ID',
                            'Tm1Score' : 'Tm2Score', 
                            'Tm2Score' : 'Tm1Score'
                            }, inplace = True)
all_games_c['Tm1Loc'] = all_games_c['Tm1Loc'].replace({'H': 'A', 'A':'H'})
all_games_f = pd.concat([all_games, all_games_c])

In [43]:
""" 
==============
A. Sanity checks
==============
"""
def checkFlips(all_games, all_games_c) -> bool:
    a = all_games.groupby(['Tm1ID'], as_index = False).sum()[['Tm1ID','wins']]
    b = all_games.groupby(['Tm2ID'], as_index = False).sum()[['Tm2ID','wins']]
    c = all_games.groupby(['Tm2ID'], as_index = False).sum()[['Tm2ID','wins']]
    d = all_games.groupby(['Tm1ID'], as_index = False).sum()[['Tm1ID','wins']]

    e = a.merge(b, suffixes = ['_W','_L'], left_on = 'Tm1ID', right_on = 'Tm2ID')
    f = c.merge(d, suffixes = ['_L','_W'], left_on = 'Tm2ID', right_on = 'Tm1ID')
    e.merge(f, on = ['Tm1ID','Tm2ID','wins_W','wins_L'])
    assert len(e) == len(a)
    return True

def checkHomeAway(all_games, all_games_c) -> bool:
    a = all_games['Tm1Loc'].value_counts()
    b = all_games_c['Tm1Loc'].value_counts()

    for i in range(len(a)):
        assert a[i] == b[i]
    return True

def checkNoDups(all_games_f) -> bool:
    assert all(all_games_f == all_games_f.drop_duplicates())
    return True

assert (checkFlips(all_games, all_games_c) and checkHomeAway(all_games, all_games_c)) and checkNoDups(all_games_f)

#### 1b. Merge team names on

In [44]:
"""
=================
A. Merge team names
=================
"""
def mergeTeamNames(df, teams):
    int1 = df.merge(teams[['TeamID','TeamName']], left_on = 'Tm1ID', right_on = 'TeamID').drop(columns='TeamID',axis=1)
    int1 = int1.merge(teams[['TeamID','TeamName']], left_on = 'Tm2ID', right_on = 'TeamID').drop(columns='TeamID',axis=1)
    return int1

int1 = mergeTeamNames(all_games_f, teams)
int1.head()

,Season,DayNum,Tm1ID,Tm1Score,Tm2ID,Tm2Score,Tm1Loc,NumOT,wins,CRType,CityID,has_city,City,State,TeamName_x,TeamName_y
0,1985,20,1228.0,81.0,1328.0,64.0,N,0.0,1.0,NaN,NaN,0,NaN,NaN,Illinois,Oklahoma
1,1985,33,1228.0,73.0,1328.0,70.0,H,0.0,1.0,NaN,NaN,0,NaN,NaN,Illinois,Oklahoma
2,1986,33,1228.0,57.0,1328.0,59.0,N,0.0,1.0,NaN,NaN,0,NaN,NaN,Illinois,Oklahoma
3,1995,67,1106.0,68.0,1328.0,97.0,N,0.0,1.0,NaN,NaN,0,NaN,NaN,Alabama St,Oklahoma
4,1990,82,1112.0,78.0,1328.0,74.0,H,0.0,1.0,NaN,NaN,0,NaN,NaN,Arizona,Oklahoma


#### 1c. Join Massey Ordinals

##### i. Select rating systems to use - which systems are from 2011-2022 with good data availability

In [45]:
END_YEAR = 2022

"""
=================
A. Decide scoring systems to keep
=================
"""
def keepScoringSystems(ords) -> pd.DataFrame:
    # 1. Get min, max ranking day for each scoring system
    ord_sub_int = ords.groupby(['SystemName','Season'], as_index = False).agg({'RankingDayNum' : [min, max]})
    ord_sub_int.columns = ['SystemName','Season','min','max']
    ord_sub_int = ord_sub_int.pivot_table(index = ['SystemName'], columns = 'Season', values = ['min', 'max'])
    ord_sub_int.columns = [i[0] + '_' + str(i[1]) for i in ord_sub_int.columns]
    ord_sub_int = ord_sub_int.reset_index()

    # 2. Re-arrange columns to go by year
    empty = []
    for i in range(2003, END_YEAR + 1):
        empty.append('min_' + str(i))
        empty.append('max_' + str(i))
    ord_sub_int = ord_sub_int[['SystemName'] + empty]

    # 3. View only systems that
    #   - a. Had ratings in 2011 and 2022
    #   - b. Had their first ratings 2011 and 2022 prior to day 23 of the season
    ord_sub = ord_sub_int.loc[(ord_sub_int['min_2022'].notnull()) & (ord_sub_int['min_2022'] < 23) & 
                (ord_sub_int['min_2011'].notnull()) & (ord_sub_int['min_2011'] < 23),
                ['SystemName'] + [i for i in empty if (int(i[-4:]) in range(2011, 2023) and i[:3] == 'min')]].copy()

    # 4. Keep systems with first ratings within first three weeks of season
    #  Barring 2021 - (COVID-impacted year)
    print(ord_sub)
    ord_sub['max_day'] = ord_sub.drop('min_2021', axis=1).max(1, numeric_only = True)
    return ord_sub[ord_sub['max_day'] < 23]

"""
=================
b. Remove top-25 polls
=================
"""
def rmTop25Polls(ords : pd.DataFrame, systems : List[str]) -> List[str]:
    tmsRanked = ords[ords['SystemName'].isin(systems)].groupby(['Season','SystemName','RankingDayNum'], as_index = False).count()
    tmsRanked = tmsRanked.groupby(['Season', 'SystemName'], as_index = False).min()[['Season','SystemName','TeamID']]
    tmsRanked = tmsRanked.pivot_table(index = 'SystemName', columns = 'Season', values = 'TeamID').reset_index()
    return list(tmsRanked.loc[tmsRanked[2011] > 25, 'SystemName'])

# Decide which scoring systems you want to keep based on 
# availability of data going back by years
systems = keepScoringSystems(ords)
systemNames = list(systems['SystemName'])

# Limit those systems further by eliminating those that
# don't rank every team in the country
kpSystems = rmTop25Polls(ords, systemNames)


# Limit ords to identified systems
ords_i = ords[(ords['SystemName'].isin(kpSystems)) & (ords['Season'] >= 2011)]
ords_i.head()

    SystemName  min_2011  min_2012  min_2013  min_2014  min_2015  min_2016  \
3           AP       9.0       7.0       8.0       9.0      16.0      16.0   
43         DES      13.0      13.0      16.0       9.0      23.0      23.0   
45         DOK       8.0      15.0       8.0       9.0      16.0      16.0   
97         MAS       0.0      12.0       7.0       9.0      16.0      16.0   
105        MOR       9.0       7.0       8.0       9.0      16.0      16.0   
117        PGH       9.0       7.0       8.0       9.0      16.0      16.0   
123        POM       9.0       7.0       8.0       9.0      16.0      16.0   
144        SAG      15.0      11.0       8.0       9.0      16.0      16.0   
176        USA       9.0       7.0       8.0       9.0      16.0      16.0   

     min_2017  min_2018  min_2019  min_2020  min_2021  min_2022  
3        16.0      16.0       9.0       9.0      30.0      16.0  
43       23.0      23.0      16.0       9.0      30.0      16.0  
45       16.0      16

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
1256303,2011,0,MAS,1102,233
1256304,2011,0,MAS,1103,109
1256305,2011,0,MAS,1104,64
1256306,2011,0,MAS,1105,332
1256307,2011,0,MAS,1106,278


In [46]:
print("This is subset of systems we want to keep")
print("From here, we also remove AP and USA, which only rank top 25 geams")
systems

This is subset of systems we want to keep
From here, we also remove AP and USA, which only rank top 25 geams


,SystemName,min_2011,min_2012,min_2013,min_2014,min_2015,min_2016,min_2017,min_2018,min_2019,min_2020,min_2021,min_2022,max_day
3,AP,9.0,7.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
45,DOK,8.0,15.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
97,MAS,0.0,12.0,7.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
105,MOR,9.0,7.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
117,PGH,9.0,7.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
123,POM,9.0,7.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
144,SAG,15.0,11.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0
176,USA,9.0,7.0,8.0,9.0,16.0,16.0,16.0,16.0,9.0,9.0,30.0,16.0,16.0


##### iii. Pick years of data to keep based on consistency of data

In [47]:
"""
=================
b. Choose season based on rk day
=================
"""

# Note 2014 is year when ratings start being published
# in a consistent weekly fashion - will be much easier 
# to work with
print(ords_i.groupby('Season', as_index = False)['RankingDayNum'].nunique())
RK_START_YEAR = 2014

    Season  RankingDayNum
0     2011             42
1     2012             35
2     2013             36
3     2014             19
4     2015             18
5     2016             18
6     2017             18
7     2018             18
8     2019             19
9     2020             18
10    2021             16
11    2022             17


##### iv. Select ranking day associated with each unique season-day in data

In [48]:
"""
====================
A. Year-RankDay Dict
====================
For years 2014 onwards, rating start coming in on same days
- So create dict of {Year : [RkDay1, RkDay2, ...]}
     -for each of those years
"""     
def rankDaysDict(ords_f : pd.DataFrame) -> Dict[int, List[int]]:
     tmp = ords_f.loc[ords_f['Season'] >= RK_START_YEAR, ['Season','RankingDayNum']].drop_duplicates()
     yrDict = {}
     for i in range(RK_START_YEAR, END_YEAR + 1):
          yrDict[i] = [0] + list(tmp.loc[tmp['Season'] == i, 'RankingDayNum'].unique()) + [500]
     return yrDict

yrDict= rankDaysDict(ords_i)

"""
====================
B. Rank day for each data day
====================
For each unique season-day present in the actual data, match to appropriate
 ranking data
"""
def getRkDayDf(df : pd.DataFrame, yrDict : Dict[int, List[int]]) -> pd.DataFrame:
     days_df = df.loc[(int1['Season'] >= RK_START_YEAR), ['Season', 'DayNum']].drop_duplicates().sort_values(['Season','DayNum'])
     days_df 

     def getRkDay(row):
          season = row['Season']
          day = row['DayNum']
          for num, i in enumerate(yrDict[season]):
               if i >= day:
                    return yrDict[season][num - 1]

     days_df['RankingDayNum'] = days_df.apply(getRkDay, axis = 1)
     return days_df

rkDayDf = getRkDayDf(int1, yrDict)
assert (rkDayDf['DayNum'] < rkDayDf['RankingDayNum']).sum() == 0

##### v. Create final ords dataset

In [49]:
"""
====================
A. Pivot ordinals 
====================
"""
ords_f = ords_i[ords_i['Season'] >= RK_START_YEAR]
ords_f = ords_f.pivot_table(index = ['Season','RankingDayNum','TeamID'],
                    values = 'OrdinalRank', 
                    columns = 'SystemName').reset_index().sort_values(['Season','RankingDayNum','TeamID'])
# ords_f[ords_f.isnull().sum(1)  > 0]
#Note that there are rows with null data in the 2021 season

##### vi. Do big join

In [50]:
"""
===============
A. Merge rank lookup
===============
"""
def mergeRkLookup(df : pd.DataFrame) -> pd.DataFrame:
    """
    Merges rank day lookup onto working dataframe
    """
    int2 = df.merge(rkDayDf, on = ['Season','DayNum'], how ='left', indicator = True)
    int2 = int2[int2['_merge'] == 'both'].sort_values(['Season','DayNum'])
    assert len(int2) == len(int1[int1['Season'] >= 2014]) # Check merge captures all data 2014 onwards
    int2.drop('_merge',axis = 1, inplace= True)
    return int2
int2 = mergeRkLookup(int1)

"""
===============
B. Merge ranks
===============
"""
def mergeRanks(df : pd.DataFrame, ords_f : pd.DataFrame) -> pd.DataFrame:
    # 1. Merge ranks by Tm1 
    # -(outer join to do sanity checks below)
    tmp = int2.merge(ords_f, left_on = ['Season','RankingDayNum','Tm1ID'],
                        right_on = ['Season','RankingDayNum','TeamID'],
                        how = 'left',
                        indicator = True).drop('TeamID', axis = 1)
    
    # Sanity checks on what data is excluded
    # Excluded data is that which has DayNum occuring before first RankingDayNum
    print("Max value of DayNum that is ultimately excluded from dataset for each year...")
    print(tmp[tmp['_merge'] == 'left_only'].groupby(['Season']).max(numeric_only = True)['DayNum'].reset_index())

    print("\nCompare to min day of rankings as we saw in systems dataset above")
    system_show_cols = [i for i in systems.columns if i[-5 : -4]  == '_' and int(i[-4:]) >= 2014]
    print(systems[system_show_cols].max())

    # Now that point is made, exclude DayNums without valid RankingDayNum preceding them
    tmp = tmp[tmp['_merge'] == 'both']
    tmp.drop('_merge',axis = 1, inplace = True)
    
    # 2. Merge ranks by Tm2 
    # -(inner join b/c sanity check already proven above)
    tmp2 = tmp.merge(ords_f, left_on = ['Season','RankingDayNum','Tm2ID'],
                        right_on = ['Season','RankingDayNum','TeamID'],
                        how = 'inner',
                        suffixes = ['_tm1','_tm2']).drop('TeamID', axis = 1)
    assert len(tmp) == len(tmp2)
    return tmp2

int3 = mergeRanks(int2, ords_f)
int3

Max value of DayNum that is ultimately excluded from dataset for each year...
   Season  DayNum
0    2014       9
1    2015      16
2    2016      16
3    2017      16
4    2018      16
5    2019       9
6    2020       9
7    2021      30
8    2022      16

Compare to min day of rankings as we saw in systems dataset above
min_2014     9.0
min_2015    16.0
min_2016    16.0
min_2017    16.0
min_2018    16.0
min_2019     9.0
min_2020     9.0
min_2021    30.0
min_2022    16.0
dtype: float64


,Season,DayNum,Tm1ID,Tm1Score,Tm2ID,Tm2Score,Tm1Loc,NumOT,wins,CRType,CityID,has_city,City,State,TeamName_x,TeamName_y,RankingDayNum,DOK_tm1,MAS_tm1,MOR_tm1,PGH_tm1,POM_tm1,SAG_tm1,DOK_tm2,MAS_tm2,MOR_tm2,PGH_tm2,POM_tm2,SAG_tm2
0,2014,10,1390.0,71.0,1321.0,58.0,H,0.0,1.0,Regular,4331.0,1,Stanford,CA,Stanford,Northwestern,9.0,65.0,45.0,37.0,44.0,55.0,58.0,96.0,82.0,104.0,106.0,80.0,71.0
1,2014,13,1229.0,68.0,1321.0,64.0,A,0.0,1.0,Regular,4113.0,1,Evanston,IL,Illinois St,Northwestern,9.0,137.0,89.0,213.0,86.0,178.0,121.0,96.0,82.0,104.0,106.0,80.0,71.0
2,2014,16,1227.0,58.0,1321.0,93.0,H,0.0,1.0,Regular,4071.0,1,Chicago,IL,IL Chicago,Northwestern,9.0,233.0,210.0,242.0,201.0,263.0,196.0,96.0,82.0,104.0,106.0,80.0,71.0
3,2014,10,1427.0,62.0,1222.0,80.0,A,0.0,1.0,Regular,4157.0,1,Houston,TX,UT San Antonio,Houston,9.0,276.0,292.0,301.0,291.0,304.0,252.0,171.0,170.0,193.0,145.0,183.0,125.0
4,2014,13,1250.0,66.0,1222.0,80.0,A,0.0,1.0,Regular,4157.0,1,Houston,TX,Lehigh,Houston,9.0,118.0,148.0,181.0,120.0,189.0,165.0,171.0,170.0,193.0,145.0,183.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88491,2022,131,1463.0,67.0,1335.0,61.0,N,0.0,1.0,Regular,4039.0,1,Boston,MA,Yale,Penn,128.0,160.0,138.0,149.0,146.0,147.0,135.0,205.0,181.0,255.0,211.0,213.0,186.0
88492,2022,131,1335.0,61.0,1463.0,67.0,N,0.0,1.0,Regular,4039.0,1,Boston,MA,Penn,Yale,128.0,205.0,181.0,255.0,211.0,213.0,186.0,160.0,138.0,149.0,146.0,147.0,135.0
88493,2022,132,1343.0,64.0,1463.0,66.0,N,0.0,1.0,Regular,4052.0,1,Cambridge,MA,Princeton,Yale,128.0,127.0,115.0,87.0,100.0,106.0,120.0,160.0,138.0,149.0,146.0,147.0,135.0
88494,2022,131,1343.0,77.0,1165.0,73.0,N,0.0,1.0,Regular,4039.0,1,Boston,MA,Princeton,Cornell,128.0,127.0,115.0,87.0,100.0,106.0,120.0,233.0,190.0,176.0,210.0,195.0,205.0


#### 1d. Merge Game and Team Cities

In [52]:
"""
===============
A. Get city for each team
===============
"""
geolocator = Nominatim(user_agent="geoapiExercises")
def getTeamCity(df):
    res = int3.groupby(['Tm1ID','TeamName_x','City','State']).count()[['Season']].reset_index().sort_values(['Tm1ID','Season'],
                    ascending = [True, False])
    res_n = res.drop_duplicates(['Tm1ID','TeamName_x'])
    res_n = res_n.drop('Season',axis = 1)
    res_n.columns = ['TmID','TeamName','City','State']
    return res_n

# tmCities_p = getTeamCity(int3)

"""
===============
B. Get coordinates for each team-city
===============
"""
def getCityCoords(row):
    if' lat' in row and row['lat'].notnull():
        return row
    loc = row['City'] + ', ' + row['State']
    try:
        res = geolocator.geocode(loc)
    except:
        return row
    if res:
        row['lat'] = res.latitude
        row['long'] = res.longitude
    return row
"""
===============
C. Save / re-load team-city coords
===============
"""
# Skip coordinate-retrieval, have them saved
# teamCities = tmCities_p.apply(getCityCoords, axis = 1)
# with open('../data/my_data/teamLocs.p', 'wb') as handle:
    # pickle.dump(teamCities, handle, protocol=pickle.HIGHEST_PROTOCOL)

# And load dataset from saved location
with open('../data/my_data/teamLocs.p', 'rb') as handle:
    teamCities = pickle.load(handle).rename(columns = {'TeamName' : 'TmName',
                            'City' : 'TmCity',
                            'State' : 'TmState',
                            'lat' : 'TmLat',
                            'long' : 'TmLong'})

# def fixCities(teamCities):
#     tmp = geolocator.geocode('Villanova, Pennsylvania')
#     teamCities.loc[teamCities['TmName']  == 'Villanova', 'TmLat'] = tmp.latitude
#     teamCities.loc[teamCities['TmName']  == 'Villanova', 'TmLong'] = tmp.longitude

#     tmp = geolocator.geocode('Chicago, Illinois')
#     teamCities.loc[teamCities['TmName']  == 'DePaul', 'TmCity'] = 'Chicago'
#     teamCities.loc[teamCities['TmName']  == 'DePaul', 'TmLat'] = tmp.latitude
#     teamCities.loc[teamCities['TmName']  == 'DePaul', 'TmLong'] = tmp.longitude

#     tmp = geolocator.geocode('Columbia, Missouri')
#     teamCities.loc[teamCities['TmName']  == 'Missouri', 'TmCity'] = 'Columbia'
#     teamCities.loc[teamCities['TmName']  == 'Missouri', 'TmState'] = 'MO'
#     teamCities.loc[teamCities['TmName']  == 'Missouri', 'TmLat'] = tmp.latitude
#     teamCities.loc[teamCities['TmName']  == 'Missouri', 'TmLong'] = tmp.longitude
#     return teamCities
# teamCities = fixCities(teamCities)
teamCities.head()

,TmID,TmName,TmCity,TmState,TmLat,TmLong
0,1101.0,Abilene Chr,Abilene,TX,32.446450,-99.747591
83,1102.0,Air Force,USAF Academy,CO,38.774069,-104.301534
85,1103.0,Akron,Akron,OH,41.083064,-81.518485
161,1104.0,Alabama,Tuscaloosa,AL,33.209561,-87.567526
206,1105.0,Alabama A&M,Normal,AL,34.788979,-86.571937


In [53]:
"""
================
A. Get game city coords
================
"""
# gameCities = int3[['CityID','City','State']].drop_duplicates()
# gameCities = gameCities.merge(teamCities[['TmCity','TmState','TmLat','TmLong']], 
#                                 left_on = ['City','State'], 
#                                 right_on = ['TmCity','TmState'],
#                                 how = 'left')
# gmCities = gameCities.apply(getCityCoords,axis = 1)
# gmCities

"""
Fill in Cayman Islands game
"""
# geolocator = Nominatim(user_agent="geoapiExercises")
# gci = geolocator.geocode('Grand Cayman Islands')
# gmCities.loc[gmCities['City']== 'Grand Cayman', 'lat'] = gci.latitude
# gmCities.loc[gmCities['City']== 'Grand Cayman', 'long'] = gci.longitude


"""
================
B. Save out and re-load
================
"""
# with open('../data/my_data/gameLocs.p', 'wb') as handle:
    # pickle.dump(gameCities, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../data/my_data/gameLocs.p', 'rb') as handle:
    gameCities = pickle.load(handle)[['City','CityID','State','lat','long']].drop_duplicates()

# def fixGameCities(gameCities):
#     gameCities.loc[gameCities['City']== ('Moon'),'lat'] = gameCities.loc[gameCities['City']== ('Moon Township'),'lat']
#     gameCities.loc[gameCities['City']== ('Moon'),'long'] = gameCities.loc[gameCities['City']== ('Moon Township'),'long']

#     vpa = geolocator.geocode('Villanova, Pennsylvania')
#     gameCities.loc[gameCities['City']  == 'Villanova', 'lat'] = vpa.latitude
#     gameCities.loc[gameCities['City']  == 'Villanova', 'long'] = vpa.longitude
#     return gameCities
gameCities.head()


,City,CityID,State,lat,long
0,Stanford,4331.0,CA,37.426541,-122.170305
1,Evanston,4113.0,IL,42.044739,-87.693046
2,Chicago,4071.0,IL,41.875562,-87.624421
5,Houston,4157.0,TX,29.758938,-95.367697
9,Iowa City,4162.0,IA,41.661256,-91.529911


In [54]:

"""
================== 
A. Merge cities to data, calc distances
================== 
"""
def calcDist(row):
    c1 = row['TmLat_tm1'], row['TmLong_tm1']
    c2 = row['TmLat_tm2'], row['TmLong_tm2']

    c_game = row['gmLat'], row['gmLong']
    row['dist_tm1'] =  geopy.distance.geodesic(c1, c_game).miles
    row['dist_tm2'] =  geopy.distance.geodesic(c2, c_game).miles
    return row

def mergeCities(df, teamCities, gameCities):
    # i. Fix game city information
    df.loc[(df['Tm1ID'] == 1281) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4463, 'Columbia', 'MO'
    df.loc[(df['Tm1ID'] == 1281) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4463, 'Columbia', 'MO'
    df.loc[(df['Tm1ID'] == 1177) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4071, 'Chicago', 'IL'
    df.loc[(df['Tm1ID'] == 1385) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4239, 'New York','NY'
    df.loc[(df['Tm1ID'] == 1163) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4149, 'Hartford','CT'
    df.loc[(df['Tm1ID'] == 1437) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4266, 'Philadelphia','PA'
    df.loc[(df['Tm1ID'] == 1155) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4074, 'Clemson','SC'
    df.loc[(df['Tm1ID'] == 1374) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4088, 'Dallas','TX'
    df.loc[(df['Tm1ID'] == 1460) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4091, 'Dayton','OH'
    df.loc[(df['Tm1ID'] == 1450) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4284, 'Pullman','WA'
    df.loc[(df['Tm1ID'] == 1108) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4200, 'Lorman','MS'
    df.loc[(df['Tm1ID'] == 1263) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4023, 'Bangor','ME'
    df.loc[(df['Tm1ID'] == 1358) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4160, 'Huntsville','TX'
    df.loc[(df['Tm1ID'] == 1116) & (df['Tm1Loc'] == 'H'), ['CityID','City','State']] = 4119, 'Fayetteville','AR'
    
    # 1. Get City informaiton on dataset
    int4 = df.merge(teamCities[['TmID','TmCity','TmState','TmLat','TmLong']], 
                            left_on = 'Tm1ID', 
                            right_on = 'TmID').drop('TmID',axis = 1)
    int4 = int4.merge(teamCities[['TmID','TmCity','TmState','TmLat','TmLong']], 
                            left_on = 'Tm2ID', 
                            right_on = 'TmID',
                            suffixes = ['_tm1','_tm2']).drop('TmID', axis = 1)
    int4 = int4.merge(gameCities[['CityID','lat','long']], on = 'CityID', how = 'left', indicator =True).rename(
                            columns = {'lat' : 'gmLat', 'long' :'gmLong', 'CityID' : 'gmCityID', 'City' : 'gmCity', 'State' : 'gmState'})
    
    assert len(int4) == len(df)
    int4 = int4[int4.isnull().sum(1) == 0]

    # 2. Calculate distances
    int4 = int4.apply(calcDist, axis = 1)

    return int4

int4 = mergeCities(int3, teamCities, gameCities)
int4

,Season,DayNum,Tm1ID,Tm1Score,Tm2ID,Tm2Score,Tm1Loc,NumOT,wins,CRType,gmCityID,has_city,gmCity,gmState,TeamName_x,TeamName_y,RankingDayNum,DOK_tm1,MAS_tm1,MOR_tm1,PGH_tm1,POM_tm1,SAG_tm1,DOK_tm2,MAS_tm2,MOR_tm2,PGH_tm2,POM_tm2,SAG_tm2,TmCity_tm1,TmState_tm1,TmLat_tm1,TmLong_tm1,TmCity_tm2,TmState_tm2,TmLat_tm2,TmLong_tm2,gmLat,gmLong,_merge,dist_tm1,dist_tm2
0,2014,10,1390.0,71.0,1321.0,58.0,H,0.0,1.0,Regular,4331.0,1,Stanford,CA,Stanford,Northwestern,9.0,65.0,45.0,37.0,44.0,55.0,58.0,96.0,82.0,104.0,106.0,80.0,71.0,Stanford,CA,37.426541,-122.170305,Evanston,IL,42.044739,-87.693046,37.426541,-122.170305,both,0.000000,1850.981596
1,2014,13,1229.0,68.0,1321.0,64.0,A,0.0,1.0,Regular,4113.0,1,Evanston,IL,Illinois St,Northwestern,9.0,137.0,89.0,213.0,86.0,178.0,121.0,96.0,82.0,104.0,106.0,80.0,71.0,Normal,IL,40.509296,-88.984394,Evanston,IL,42.044739,-87.693046,42.044739,-87.693046,both,125.482703,0.000000
2,2014,16,1227.0,58.0,1321.0,93.0,H,0.0,1.0,Regular,4071.0,1,Chicago,IL,IL Chicago,Northwestern,9.0,233.0,210.0,242.0,201.0,263.0,196.0,96.0,82.0,104.0,106.0,80.0,71.0,Chicago,IL,41.875562,-87.624421,Evanston,IL,42.044739,-87.693046,41.875562,-87.624421,both,0.000000,12.199555
3,2015,49,1227.0,46.0,1321.0,63.0,A,0.0,1.0,Regular,4113.0,1,Evanston,IL,IL Chicago,Northwestern,44.0,288.0,292.0,286.0,276.0,299.0,307.0,141.0,122.0,88.0,116.0,132.0,142.0,Chicago,IL,41.875562,-87.624421,Evanston,IL,42.044739,-87.693046,42.044739,-87.693046,both,12.199555,0.000000
4,2022,17,1192.0,46.0,1321.0,82.0,A,0.0,1.0,Regular,4113.0,1,Evanston,IL,F Dickinson,Northwestern,16.0,320.0,336.0,314.0,307.0,327.0,316.0,62.0,43.0,29.0,64.0,42.0,54.0,Hackensack,NJ,40.887144,-74.041087,Evanston,IL,42.044739,-87.693046,42.044739,-87.693046,both,712.379408,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88491,2022,18,1464.0,79.0,1472.0,75.0,H,0.0,1.0,Regular,4373.0,1,Youngstown,OH,Youngstown St,St Thomas MN,16.0,222.0,204.0,213.0,246.0,268.0,219.0,298.0,350.0,358.0,345.0,331.0,357.0,Youngstown,OH,41.103579,-80.652016,St. Paul,MN,44.949749,-93.093103,41.103579,-80.652016,both,0.000000,682.808873
88492,2022,26,1370.0,81.0,1472.0,64.0,H,0.0,1.0,Regular,4313.0,1,Seattle,WA,Seattle,St Thomas MN,23.0,205.0,179.0,173.0,182.0,181.0,184.0,261.0,305.0,352.0,333.0,310.0,340.0,Seattle,WA,47.603832,-122.330062,St. Paul,MN,44.949749,-93.093103,47.603832,-122.330062,both,0.000000,1403.705407
88493,2022,19,1310.0,67.0,1472.0,76.0,N,0.0,1.0,Regular,4373.0,1,Youngstown,OH,Niagara,St Thomas MN,16.0,200.0,194.0,140.0,194.0,221.0,244.0,298.0,350.0,358.0,345.0,331.0,357.0,Niagara Falls,NY,43.084360,-79.061469,St. Paul,MN,44.949749,-93.093103,41.103579,-80.652016,both,159.287790,682.808873
88494,2022,82,1377.0,90.0,1472.0,79.0,A,0.0,1.0,Regular,4482.0,1,St. Paul,MN,South Dakota,St Thomas MN,79.0,253.0,237.0,207.0,189.0,247.0,240.0,278.0,284.0,271.0,280.0,257.0,310.0,Vermillion,SD,42.779527,-96.929101,St. Paul,MN,44.949749,-93.093103,44.949749,-93.093103,both,243.190874,0.000000


In [55]:
"""
=============
A. Help check which games seem to have wrong locations
=============
"""
# Are home teams playing 'home' games far from their home city?
sub = int4.loc[(int4['Tm1Loc'] == 'H') & (int4['dist_tm1'] > 0)].sort_values('dist_tm1',ascending = False)
test = sub.groupby('TeamName_x').agg({'dist_tm1' : ['max', 'count']}).reset_index()
test.columns = ['TeamName','mean','count']
test.sort_values('count', ascending = False).head(50)

,TeamName,mean,count
50,Villanova,11.446611,113
24,Niagara,6.224197,50
31,Presbyterian,1.027453,44
17,MS Valley St,8.407085,36
44,St Francis NY,3.829933,34
43,St Bonaventure,2.622919,33
18,Manhattan,3.994730,33
1,Alabama A&M,4.152825,29
35,Robert Morris,12.322248,25
4,Ark Little Rock,2.040392,19


In [444]:
# with open('../data/my_data/ds1.p', 'wb') as handle:
#     pickle.dump(int4, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('../data/my_data/gameLocs.p', 'rb') as handle:
#     gameCities = pickle.load(handle)[['City','CityID','State','lat','long']].drop_duplicates()
